In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model, Sequential

import pandas as pd
import numpy as np

import random
random.seed(1228)

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

%matplotlib inline

Using TensorFlow backend.


### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

#### Reading the data:

In [2]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [3]:
columns = ['ignore', 'words', 'ignore', 'pos', 'chunk']
train_reader = ConllCorpusReader(root = '.', fileids = 'unamb_sent_14_6.conllu', columntypes = columns)


sents = train_reader.iob_sents()
train_reader.iob_sents()

[[('«', 'PUNCT', '_'), ('Школа', 'NOUN', '_'), ('злословия', 'NOUN', '_'), ('»', 'PUNCT', '_'), ('учит', 'VERB', '_'), ('прикусить', 'VERB', '_'), ('язык', 'NOUN', '_')], [('Сохранится', 'VERB', '_'), ('ли', 'PART', '_'), ('градус', 'NOUN', '_'), ('дискуссии', 'NOUN', '_'), ('в', 'ADP', '_'), ('новом', 'ADJ', '_'), ('сезоне', 'NOUN', '_'), ('?', 'PUNCT', '_')], ...]

#### Считаем количество предложений и разных тэгов

In [4]:
len(sents) # Предложений в корпусе 38508

38508

In [5]:
#вытащим все тэги
pos = [w[1] for sent in sents for w in sent]
pos[:15]

['PUNCT',
 'NOUN',
 'NOUN',
 'PUNCT',
 'VERB',
 'VERB',
 'NOUN',
 'VERB',
 'PART',
 'NOUN',
 'NOUN',
 'ADP',
 'ADJ',
 'NOUN',
 'PUNCT']

In [6]:
# вот столько у нас разных тэгов
len(set(pos))

14

In [7]:
# а теперь посмотрим на частотное распределение тэгов 
from nltk import FreqDist
fd_pos = FreqDist(pos)

In [8]:
fd_pos

FreqDist({'ADJ': 47487,
          'ADP': 42835,
          'ADV': 13079,
          'CONJ': 21942,
          'DET': 12689,
          'INTJ': 452,
          'NOUN': 121793,
          'NUM': 10173,
          'PART': 8923,
          'PRON': 9067,
          'PROPN': 14889,
          'PUNCT': 91323,
          'VERB': 41538,
          'X': 21393})

Как мы видим, самые частые -- прилагательные и предлоги (о_О)

#### Сформируем тестовую и тренировочную выборки для тэгов

Итак, вектор POS у нас есть: `pos`. Теперь сделаем вектор признаков. Для удобства мы решили убрать информацию о границах предложений.

In [9]:
words = [word[0].lower() for sent in sents for word in sent]
words[:20]

['«',
 'школа',
 'злословия',
 '»',
 'учит',
 'прикусить',
 'язык',
 'сохранится',
 'ли',
 'градус',
 'дискуссии',
 'в',
 'новом',
 'сезоне',
 '?',
 'великолепная',
 '«',
 'школа',
 'злословия',
 '»']

Извлечение признаков:

In [10]:
embeddingsPath = './wiki.ru.vec'

In [11]:
def extract_embeddings(path):
    embeddings = {}
    with open(embeddingsPath) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            embeddings[word] = [float(num) for num in vec.split()]
    return embeddings

embeddings = extract_embeddings(embeddingsPath)

In [12]:
embeddings['школа'][:15]

[-0.32315,
 0.91456,
 0.13797,
 0.61075,
 -0.28406,
 -0.47918,
 -0.27341,
 0.17947,
 0.54726,
 -0.47914,
 -0.20418,
 0.12833,
 0.1399,
 0.26005,
 0.53394]

In [13]:
len(embeddings['школа'])

300

In [14]:
# a strange fix
embeddings['«'] = embeddings['"']
embeddings['»'] = embeddings['"']

In [18]:
# создаём вектора для начала и конца (и плэйсхолдер на случай если в эмбеддингах слова нет)
MOF = [0] * 300

In [24]:
# k == 2
def featues_k2(tokens):
    features = []
    for i, word in enumerate(tokens):
        if word in embeddings:
            cur_word = embeddings[word]
        else:
            cur_word = MOF
        if i > 0 and tokens[i - 1] in embeddings:
            prev_word = embeddings[tokens[i - 1]]
        else:
            prev_word = MOF
        if i < len(tokens) - 1 and tokens[i + 1] in embeddings:
            next_word = embeddings[tokens[i + 1]]
        else:
            next_word = MOF
        features.append(cur_word + prev_word + next_word)
    return features

In [26]:
features = featues_k2(words)

In [29]:
len(features) == len(pos)

True

In [30]:
boundary = (len(pos) // 4) * 3
y_train = pos[:boundary]
X_train = features[:boundary]
y_test = pos[boundary:]
X_test = features[boundary:]

print(len(y_train))
print(len(y_test))

343185
114398


### Часть 2 [4 баллов] Архитектура нейронной сети

Архитектура нейронной сети состоит из следующих слов:
1. Входной слой: нейронная сеть получает на вход вектор признаков, состоящий из $k$ конкатенированных эмбеддингов;/
2. Скрытый слой: $n_h$ нейронов и нелинейная функция активации $\theta$;
3. Выходной слой:  $|T|$ нейронов для итоговой классификации.

Обучите нейронную сеть на обучающих данных.

я бы поставила семь, конечно, но можно и три пока а это для последнего задания

In [ ]:
TEXT_LENGTH = 1800
VOCABULARY_SIZE = 250000
EMBEDDING_DIM = 300
DIMS = 250
MAX_FEATURES = 5000
batch_size = 32

nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 10

In [42]:
sequences = tokenizer.texts_to_sequences(train_s.body)
X_train = tokenizer.sequences_to_matrix(sequences, mode='count')
sequences = tokenizer.texts_to_sequences(test_corpus.body)
X_test = tokenizer.sequences_to_matrix(sequences, mode='count')

NameError: name 'train_s' is not defined

Посмотреть какая именно архитектура сенны

In [ ]:
model = Sequential()
model.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, input_length=TEXT_LENGTH))
model.add(Flatten())
model.add(Dense(5, activation = 'softmax'))
model.add(Dropout(0.5))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size,  validation_split=0.1)